In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
df = pd.read_csv('3 gas_data.csv', delimiter=";")

In [4]:
df.head()

,timestamp (s),valore % gas,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,59,11.73,07:45,Chieti,04/07/2024
1,119,11.24,NaN,NaN,NaN
2,179,11.05,NaN,NaN,NaN
3,239,11.05,NaN,NaN,NaN
4,299,10.95,NaN,NaN,NaN


In [5]:
df = df.rename(columns={"valore % gas": "gas percentage", "Unnamed: 2": "time", "Unnamed: 3": "location", "Unnamed: 4": "date"})

In [6]:
df

,timestamp (s),gas percentage,time,location,date
0,59,11.73,07:45,Chieti,04/07/2024
1,119,11.24,NaN,NaN,NaN
2,179,11.05,NaN,NaN,NaN
3,239,11.05,NaN,NaN,NaN
4,299,10.95,NaN,NaN,NaN
...,...,...,...,...,...
331,3839,8.31,NaN,NaN,NaN
332,3899,8.50,NaN,NaN,NaN
333,3959,8.50,NaN,NaN,NaN
334,4019,8.31,NaN,NaN,NaN


In [7]:
df.isna().sum()

timestamp (s)       0
gas percentage      0
time              332
location          332
date              332
dtype: int64

In [8]:
df['location'] = df['location'].ffill()

In [9]:
df['date'] = df['date'].ffill()

In [10]:
df.iloc[100:110]

,timestamp (s),gas percentage,time,location,date
100,1619,9.78,NaN,San Biase,04/07/2024
101,1679,9.78,NaN,San Biase,04/07/2024
102,1739,9.68,NaN,San Biase,04/07/2024
103,1799,9.78,NaN,San Biase,04/07/2024
104,1859,9.68,NaN,San Biase,04/07/2024
105,1919,9.68,NaN,San Biase,04/07/2024
106,1979,9.78,NaN,San Biase,04/07/2024
107,2039,9.78,NaN,San Biase,04/07/2024
108,2099,9.68,NaN,San Biase,04/07/2024
109,2159,9.29,NaN,San Biase,04/07/2024


In [11]:
df.shape

(336, 5)

In [12]:
df = df[df['timestamp (s)'] != "timestamp (s)"]

In [13]:
df.shape

(333, 5)

In [14]:
df.iloc[100:110]

,timestamp (s),gas percentage,time,location,date
101,1679,9.78,NaN,San Biase,04/07/2024
102,1739,9.68,NaN,San Biase,04/07/2024
103,1799,9.78,NaN,San Biase,04/07/2024
104,1859,9.68,NaN,San Biase,04/07/2024
105,1919,9.68,NaN,San Biase,04/07/2024
106,1979,9.78,NaN,San Biase,04/07/2024
107,2039,9.78,NaN,San Biase,04/07/2024
108,2099,9.68,NaN,San Biase,04/07/2024
109,2159,9.29,NaN,San Biase,04/07/2024
110,2219,9.78,NaN,San Biase,04/07/2024


In [15]:
df.dtypes

timestamp (s)     object
gas percentage    object
time              object
location          object
date              object
dtype: object

In [16]:
df['gas percentage'] = df['gas percentage'].astype(str).astype(float)

In [17]:
df['timestamp (s)'] = df['timestamp (s)'].astype(str).astype(float)

In [22]:
df['date'] = pd.to_datetime(df['date'],format="%d/%m/%Y")

In [23]:
df['time'] = pd.to_datetime(df['time'])

In [24]:
df['time_filled'] = df['time'].fillna(method='ffill')

In [25]:
from datetime import datetime, timedelta

def compute_new_time(row):
    if pd.isnull(row['time']):
        return row['time_filled'] + timedelta(seconds=row['timestamp (s)'])
    return row['time']

df['new_time'] = df.apply(compute_new_time, axis=1)


In [26]:
df['new_time'] = df['new_time'].dt.time

In [27]:
df.drop(columns=['time','time_filled','timestamp (s)'], inplace=True)

In [28]:
df = df.rename(columns={'new_time': 'time'})

In [29]:
location_id = {
    'Guardiagrele': '001',
    'Chieti': '002',
    'San Biase': '003',
    'Monopoli': '004'    
}

df['location_id'] = df['location'].map(location_id)

In [30]:
df = df.reindex(columns=['date', 'time', 'location_id', 'location', 'gas percentage'])

In [31]:
df

,date,time,location_id,location,gas percentage
0,2024-07-04,07:45:00,002,Chieti,11.73
1,2024-07-04,07:46:59,002,Chieti,11.24
2,2024-07-04,07:47:59,002,Chieti,11.05
3,2024-07-04,07:48:59,002,Chieti,11.05
4,2024-07-04,07:49:59,002,Chieti,10.95
...,...,...,...,...,...
331,2024-07-12,20:28:59,001,Guardiagrele,8.31
332,2024-07-12,20:29:59,001,Guardiagrele,8.50
333,2024-07-12,20:30:59,001,Guardiagrele,8.50
334,2024-07-12,20:31:59,001,Guardiagrele,8.31


In [32]:
df.to_csv('5 gas_data_cleaned.csv')

In [33]:
data_locations = {
    'location_id': ['001','002','003','004'],
    'location_name': ['Guardiagrele','Chieti','San Biase','Monopoli'],
    'inhabitants': [8317, 50770, 165, 49916],
    'number of firms': [766, 4140, 11, 3547],
    'number of vehicles': [9136, 46509, 152, 42493]
}

In [34]:
df_2 = pd.DataFrame(data_locations)

In [35]:
df_2.to_csv('locations.csv')